In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

# ========================================
# 경로 설정
# ========================================
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/cifar10_data'  # CIFAR-10 데이터를 저장할 경로

# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')

if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")

# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}\n")

# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

# 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[y[0]]for y in y_train])
y_test = np.array([label_map[y[0]]for y in y_test])

# 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

데이터가 없습니다. /content/drive/MyDrive/Colab Notebooks/cifar10_data에 다운로드합니다...
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



#CNN

In [3]:
from abc import ABC
model = models.Sequential()
model.add((layers.Conv2D(32,(3,3),activation='relu',input_shape = (32,32,3))))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(3,activation='softmax')) # Changed output layer to 3 for the selected classes

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,115 (477.01 KB)

 Trainable params: 122,115 (477.01 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.4590 - loss: 1.0003
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6468 - loss: 0.7587
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6986 - loss: 0.6595
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7344 - loss: 0.6047
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7527 - loss: 0.5632


DNN

In [9]:
dnn = models.Sequential([layers.Flatten(input_shape=(32, 32, 3)),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')]) # Changed output layer to 3 for the selected classes

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
dnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [11]:
history_dnn = dnn.fit(x_train, y_train, epochs=10,
                            validation_split=0.1, batch_size=128, verbose=2)

Epoch 1/10
106/106 - 5s - 49ms/step - accuracy: 0.4109 - loss: 1.1185 - val_accuracy: 0.4707 - val_loss: 1.0008
Epoch 2/10
106/106 - 0s - 3ms/step - accuracy: 0.4664 - loss: 0.9947 - val_accuracy: 0.4887 - val_loss: 0.9832
Epoch 3/10
106/106 - 1s - 7ms/step - accuracy: 0.4796 - loss: 0.9835 - val_accuracy: 0.5360 - val_loss: 0.9409
Epoch 4/10
106/106 - 0s - 5ms/step - accuracy: 0.4956 - loss: 0.9616 - val_accuracy: 0.5227 - val_loss: 0.9477
Epoch 5/10
106/106 - 1s - 5ms/step - accuracy: 0.5033 - loss: 0.9510 - val_accuracy: 0.5580 - val_loss: 0.9140
Epoch 6/10
106/106 - 1s - 5ms/step - accuracy: 0.5184 - loss: 0.9354 - val_accuracy: 0.5407 - val_loss: 0.9538
Epoch 7/10
106/106 - 0s - 4ms/step - accuracy: 0.5259 - loss: 0.9284 - val_accuracy: 0.5287 - val_loss: 0.9212
Epoch 8/10
106/106 - 0s - 4ms/step - accuracy: 0.5136 - loss: 0.9428 - val_accuracy: 0.5327 - val_loss: 0.9341
Epoch 9/10
106/106 - 0s - 4ms/step - accuracy: 0.5176 - loss: 0.9329 - val_accuracy: 0.5213 - val_loss: 0.9442


In [12]:
test_loss, test_acc = dnn.evaluate(x_test, y_test, verbose=2)
print(f"DNN Test Accuracy: {test_acc:.4f}")

94/94 - 1s - 10ms/step - accuracy: 0.5400 - loss: 0.9170
DNN Test Accuracy: 0.5400
